In [36]:
from flashtext import KeywordProcessor
import pandas as pd
import os
import gensim
import logging
import datetime
import csv
import numpy as np
logging.getLogger().setLevel(logging.INFO)

from gensim.models import Word2Vec, KeyedVectors
from sklearn.feature_extraction._stop_words import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import CountVectorizer
from mittens.tf_mittens import GloVe, Mittens

In [2]:
root_dir = os.path.abspath(os.pardir)
data_dir = os.path.join(root_dir, "data")
model_dir = os.path.join(root_dir, "model")
tag_table_file = os.path.join(data_dir, "paper_tag_table.csv")
paper_table_file = os.path.join(data_dir, "paper_table.csv")
pre_trained_model = os.path.join(model_dir, "GoogleNews-vectors-negative300.bin")
output_model_file = os.path.join(model_dir, "model.bin")
glove_file = os.path.join(model_dir, "glove.6B", "glove.6B.50d.txt")

In [3]:
tag_set = set(pd.read_csv(tag_table_file)['tag_text'].dropna().tolist())
abstract_list = pd.read_csv(paper_table_file)['abstract'].dropna().tolist()[:2]

In [4]:
from phrase_word2vec import PhraseWord2VEC

In [5]:
obj = PhraseWord2VEC()
# obj.train(tag_set, abstract_list, pre_trained_model, output_model_file)

In [6]:
pre_processed_sent = obj.pre_process(tag_set, abstract_list)

INFO:root:pre processing input sentences 


In [10]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

In [11]:
embedding_dict = glove2dict(glove_file)

In [19]:
word_list = []
oov_word_set = set()

for list_ in pre_processed_sent:
    for word in list_:
        word_list.append(word)
        if word not in embedding_dict.keys():
            oov_word_set.add(word)

In [44]:
mittens_model = Mittens(n=50, max_iter=1000)
new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=list(oov_word_set),
    initial_embedding_dict= embedding_dict)

Iteration 210: loss: 0.00010739189747255296

In [50]:
with open('test.csv', 'a', newline='') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    spamwriter.writerow([list(oov_word_set)[1]] + list(new_embeddings[1]))